In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pyparsing>=2.0.2 in c:\users\enhe_\anaconda3\envs\pythondata\lib\site-packages (from packaging->bleach->nbconvert->notebook>=4.4.1->widgetsnbextension~=3.5.0->ipywidgets>=7.0.0->gmaps) (2.4.7)



In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key

In [3]:
# get that csv, then add heat map. 
cities = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,port elizabeth,-33.9180,25.5701,52.11,93,0,2.30,ZA,1624422912
1,new norfolk,-42.7826,147.0587,51.93,80,100,1.54,AU,1624422914
2,oktyabrskoye,43.0645,44.7417,75.34,73,0,2.24,RU,1624422915
3,kapaa,22.0752,-159.3190,84.18,77,40,3.00,US,1624422916
4,san rafael,-34.6177,-68.3301,47.70,72,64,4.23,AR,1624422919
...,...,...,...,...,...,...,...,...,...
555,ilongero,-4.6667,34.8667,57.04,77,7,11.14,TZ,1624423659
556,uruzgan,32.8333,66.0000,82.13,10,0,1.79,AF,1624423661
557,markova,48.6758,24.4138,63.99,80,12,1.23,UA,1624423662
558,bambanglipuro,-7.9500,110.2833,67.60,77,76,10.71,ID,1624423664


In [4]:
humidity = cities["Humidity"].astype(float)
print(humidity)
maxhumidity = humidity.max()
print(maxhumidity)
locations = cities[["Lat", "Lng"]]
print(locations)

0      93.0
1      80.0
2      73.0
3      77.0
4      72.0
       ... 
555    77.0
556    10.0
557    80.0
558    77.0
559    74.0
Name: Humidity, Length: 560, dtype: float64
100.0
         Lat       Lng
0   -33.9180   25.5701
1   -42.7826  147.0587
2    43.0645   44.7417
3    22.0752 -159.3190
4   -34.6177  -68.3301
..       ...       ...
555  -4.6667   34.8667
556  32.8333   66.0000
557  48.6758   24.4138
558  -7.9500  110.2833
559  40.8974    9.1035

[560 rows x 2 columns]


In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
 # 10 cities and do narrow down between 80-90 
    # each cities coordinates
narrowed_city_df = cities.loc[(cities["Max Temp"] < 85) & (cities["Max Temp"] > 82) & (cities["Cloudiness"] >= 0) & (cities["Wind Speed"] < 4), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kapaa,22.0752,-159.3190,84.18,77,40,3.00,US,1624422916
1,georgetown,5.4112,100.3354,84.88,75,20,1.01,MY,1624422855
2,meulaboh,4.1363,96.1285,84.99,58,92,0.89,ID,1624423077
3,delijan,33.9905,50.6838,84.96,7,0,1.72,IR,1624423290
4,batagay,67.6333,134.6333,82.31,22,69,2.15,RU,1624423499
5,rudsar,37.1376,50.2880,82.45,39,0,3.44,IR,1624423511
6,uruzgan,32.8333,66.0000,82.13,10,0,1.79,AF,1624423661


In [12]:
narrowed_city_df.loc[0]['Lat']


22.0752

In [13]:
# fig = gmaps.figure(confi )
# once you created u don't nd to recreate redo it
hotel_list=[]
for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    json_response = requested.json()
    try:
        hotel_list.append(json_response['results'][0]['name'])
    except:
        hotel_list.append("")
narrowed_city_df["Hotel Name"] = hotel_list
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,kapaa,22.0752,-159.3190,84.18,77,40,3.00,US,1624422916,Kapaʻa
1,georgetown,5.4112,100.3354,84.88,75,20,1.01,MY,1624422855,George Town
2,meulaboh,4.1363,96.1285,84.99,58,92,0.89,ID,1624423077,Meulaboh
3,delijan,33.9905,50.6838,84.96,7,0,1.72,IR,1624423290,Delijan
4,batagay,67.6333,134.6333,82.31,22,69,2.15,RU,1624423499,Batagay


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]



In [10]:
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))